## 깃허브 통해서 다운로드 받는 코드도 이해하면 좋을듯 함...!!!

In [1]:
import os
import gc
import sys
import json
import glob
import random
from pathlib import Path

import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import itertools
from tqdm import tqdm

from imgaug import augmenters as iaa
from sklearn.model_selection import StratifiedKFold, KFold

In [2]:
# 현재 폴더 위치에서 input, working 폴더를 만들어서 읽는 형태
DATA_DIR = Path('./input')
ROOT_DIR = Path('./working')

NUM_CATS = 46
IMAGE_SIZE = 512

In [3]:
# 이 부분에서 str 형태로 로드가 안되는듯함...
MASK_RCNN_PATH = str(ROOT_DIR/'Mask_RCNN')

from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log
from tensorflow import keras

ModuleNotFoundError: No module named 'keras.engine'

In [ ]:
# 깃허브 통해서 직접 다운로드
# https://github.com/matterport/Mask_RCNN/releases/download/v2.0/mask_rcnn_coco.h5

COCO_WEIGHTS_PATH = 'mask_rcnn_coco.h5'
#if not os.path.exists(COCO_WEIGHTS_PATH):
#    os.system('curl -L -o mask_rcnn_coco.h5 https://github.com/matterport/Mask_RCNN/releases/download/v2.0/mask_rcnn_coco.h5')

# 다운로드한 파일 확인
#os.system('ls -lh mask_rcnn_coco.h5')

Mask_RCNN 안의 모델을 살펴보는 것이 중요하겠다

In [ ]:
class FashionConfig(Config):
    # 설정이름을 fashion으로 정의
    NAME = "fashion"
    # NUM_CATS = 46 위에서 설정함
    NUM_CLASSES = NUM_CATS + 1
    
    GPU_COUNT = 1
    IMAGES_PER_GPU = 4

    # 백본 네트워크 설정
    BACKBONE = 'resnet50'

    # IMAGE_SIZE = 512 위에서 설정함
    IMAGE_MIN_DIM = IMAGE_SIZE
    IMAGE_MAX_DIM = IMAGE_SIZE    

    # none > 원래 크기를 유지 / square > 지정된 사이즈로 반환 + 빈공간 패딩
    # pad64 > 가장 가까운 64 배수로 리사이즈 + 패딩 / crop > 무작위로 자름
    IMAGE_RESIZE_MODE = 'none' # 이미지 리사이즈 모드는 입력 이미지를 모델에 맞게 조정하는 방법을 정의

    # 앵커 박스의 측면 길이를 설정함(정사각형 형태),  객체를 탐지하는 박스
    RPN_ANCHOR_SCALES = (16, 32, 64, 128, 256)

    # 에폭 당 스텝 수 / 한 에폭 당 배치 학습 수를 설정
    STEPS_PER_EPOCH = 1000
    # 검증 단계 스텝 수 / 한 에폭 당 배치 검증 수행 수
    VALIDATION_STEPS = 200
    
config = FashionConfig()
config.display()

In [ ]:
# json 형태 파일 로드
with open(DATA_DIR/"label_descriptions.json") as f:
    label_descriptions = json.load(f)

label_names = [x['name'] for x in label_descriptions['categories']]

In [ ]:
segment_df = pd.read_csv(DATA_DIR/"train.csv")

# 멀티라벨 데이터의 비율을 파악함 / 이미지 안에 클래스가 여러개 있는 경우
multilabel_percent = len(segment_df[segment_df['ClassId'].str.contains('_')])/len(segment_df)*100
print(f"Segments that have attributes: {multilabel_percent:.2f}%")

In [ ]:
# CategoryId 열을 추가함 / ClassId 열에서 기본 클래스만 추출하는 과정
segment_df['CategoryId'] = segment_df['ClassId'].str.split('_').str[0]

print("Total segments: ", len(segment_df))
segment_df.head()

In [ ]:
# ImageId 별로 'EncodedPixels', 'CategoryId' 열을 하나의 리스트로 저장
image_df = segment_df.groupby('ImageId')[['EncodedPixels', 'CategoryId']].agg(lambda x: list(x))
# ImageId 별로 'Height', 'Width' 값의 평균값을 저장
size_df = segment_df.groupby('ImageId')[['Height', 'Width']].mean()
# image_df, size_df ImageId 기준으로 병합
image_df = image_df.join(size_df, on='ImageId')

# 동일한 이미지에 대한 세그먼트 정보 (하나의 사진 안에 있는 여러 이미지 정보)를 하나의 데이터 프레임에 통합함
# 하나의 이미지 안에 들어있는 여러 세그먼트 정보를 동일한 데이터 프레임 안에 두어 접근 용이성을 높임
print("Total images: ", len(image_df))
image_df.head()

In [ ]:
# 이미지를 읽기 / 색상 변환 / 이미지 리사이즈 > 모델에 적합한 형태로 변환
def resize_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (IMAGE_SIZE, IMAGE_SIZE), interpolation=cv2.INTER_AREA)  
    return img

In [ ]:
# 빈 마스크 생성 > 세그먼트 정보 읽기 > 세그먼트 그리기 > 리사이즈 > 최종 마스크 생성

class FashionDataset(utils.Dataset):

    def __init__(self, df):
        super().__init__(self)
        
        # label_names 리스트를 데이터셋에 추가
        for i, name in enumerate(label_names):
            self.add_class("fashion", i+1, name)
        
        #  add_image 메서드 통해 이미지 정보를 데이터셋에 추가
        for i, row in df.iterrows():
            self.add_image("fashion", 
                           image_id=row.name, 
                           path=str(DATA_DIR/'train'/row.name), 
                           labels=row['CategoryId'],
                           annotations=row['EncodedPixels'], 
                           height=row['Height'], width=row['Width'])

    def image_reference(self, image_id):
        info = self.image_info[image_id]
        return info['path'], [label_names[int(x)] for x in info['labels']]
    
    def load_image(self, image_id):
        return resize_image(self.image_info[image_id]['path'])

    def load_mask(self, image_id):
        info = self.image_info[image_id]

        height = int(info['height'])
        width = int(info['width'])

        # mask 배열 초기화 + labels 리스트 초기화
        mask = np.zeros((IMAGE_SIZE, IMAGE_SIZE, len(info['annotations'])), dtype=np.uint8)
        labels = []

        # annotation 안에는 압축이나 픽셀화 된 숫자 형태의 이미지는 아닌 이미지 정보가 들어있고 
        # 그걸 for문을 통해 이미지 형태로 만드는 것
        # annotation, label 를 통해 세그먼트의 마스크를 생성함
        for m, (annotation, label) in enumerate(zip(info['annotations'], info['labels'])):
            sub_mask = np.full(height * width, 0, dtype=np.uint8)
            annotation = [int(x) for x in annotation.split(' ')]

            # sub_mask 배열을 재구성하고 리사이즈함 / 이미지 압축함
            # annotation[::2] RLE 인코딩의 시작 위치를 가져온다
            # 압축된 형태로 저장된 세그먼트 정보를 실제 이미지 형태로 복원할 수 있음
            for i, start_pixel in enumerate(annotation[::2]):
                # 1로 칠해라
                sub_mask[start_pixel: start_pixel+annotation[2*i+1]] = 1

            sub_mask = sub_mask.reshape((height, width), order='F')
            sub_mask = cv2.resize(sub_mask, (IMAGE_SIZE, IMAGE_SIZE), interpolation=cv2.INTER_NEAREST)
            
            mask[:, :, m] = sub_mask
            labels.append(int(label)+1)
            
        return mask, np.array(labels)

In [ ]:
dataset = FashionDataset(image_df)
dataset.prepare()

for i in range(6):
    image_id = random.choice(dataset.image_ids)
    print(dataset.image_reference(image_id))
    
    image = dataset.load_image(image_id)
    mask, class_ids = dataset.load_mask(image_id)
    visualize.display_top_masks(image, mask, class_ids, dataset.class_names, limit=4)

In [ ]:
# 전체 데이터를 5개로 나누고 / 첫 번째 폴드 선택
FOLD = 0
N_FOLDS = 5

# 데이터 무작위로 섞고 5개로 나눔
kf = KFold(n_splits=N_FOLDS, random_state=42, shuffle=True)
splits = kf.split(image_df)

def get_fold():    
    for i, (train_index, valid_index) in enumerate(splits):
        if i == FOLD:
            return image_df.iloc[train_index], image_df.iloc[valid_index]
        
train_df, valid_df = get_fold()

train_dataset = FashionDataset(train_df)
train_dataset.prepare()

valid_dataset = FashionDataset(valid_df)
valid_dataset.prepare()

In [ ]:
# 훈련 데이터와 검증 데이터에서 각 카테고리의 분포를 시각화하고, 전체 이미지와 세그먼트의 개수를 출력함

# np.concatenate 배열 합치기 / train_df['CategoryId'].values > CategoryId 모든 값 배열화
train_segments = np.concatenate(train_df['CategoryId'].values).astype(int)
print("Total train images: ", len(train_df))
print("Total train segments: ", len(train_segments))

plt.figure(figsize=(12, 3))
# np.unique > 배열의 고유한 값
values, counts = np.unique(train_segments, return_counts=True)
plt.bar(values, counts)
plt.xticks(values, label_names, rotation='vertical')
plt.show()

valid_segments = np.concatenate(valid_df['CategoryId'].values).astype(int)
print("Total train images: ", len(valid_df))
print("Total validation segments: ", len(valid_segments))

plt.figure(figsize=(12, 3))
values, counts = np.unique(valid_segments, return_counts=True)
plt.bar(values, counts)
plt.xticks(values, label_names, rotation='vertical')
plt.show()

In [ ]:
# 가중치 업데이트 비율? 속도?
LR = 1e-4
# 전체 훈련 데이터 학습하는 반복 횟수
EPOCHS = [2, 6, 8]

import warnings 
warnings.filterwarnings("ignore")

In [ ]:
# R-CNN 모델 생성 / 모델을 훈련 모드로 설정
model = modellib.MaskRCNN(mode='training', config=config, model_dir=ROOT_DIR)

# 가충치 로드 / exclude > 특정 레이어 제외 후 가중치 로드
model.load_weights(COCO_WEIGHTS_PATH, by_name=True, exclude=[
    'mrcnn_class_logits', 'mrcnn_bbox_fc', 'mrcnn_bbox', 'mrcnn_mask'])

In [ ]:
# imgaug 라이브러리 (iaa) / iaa.Sequential > 지정된 순서대로 증강 기법을 적용함
# iaa.Fliplr(0.5) > 이미지 좌우 반전 / 0.5 > 50%
# 다양한 데이터 학습할 수 있도록 도움

augmentation = iaa.Sequential([
    iaa.Fliplr(0.5)
])

In [ ]:
# %%time > 실행 시간 측정 / epochs=EPOCHS[0] > 위에서 설정한 배열 중 첫번째
# 모델의 헤드 레이어만 훈련하고 데이터 증강은 사용하지 않음

%%time
model.train(train_dataset, valid_dataset,
            learning_rate=LR*2,
            epochs=EPOCHS[0],
            layers='heads',
            augmentation=None)

history = model.keras_model.history.history

In [ ]:
# 모델의 모든 레이어 훈련함 / 데이터 증강 사용

%%time
model.train(train_dataset, valid_dataset,
            learning_rate=LR,
            epochs=EPOCHS[1],
            layers='all',
            augmentation=augmentation)

# for문을 통해 훈련 기록 누적
new_history = model.keras_model.history.history
for k in new_history: history[k] = history[k] + new_history[k]

In [ ]:
# 학습률을 다르게 설정하는 이유 > 학습률은 모델이 가중치를 업데이트하는 속도를 결정함, 최적의 학습 속도와 안정성을 찾음
# 에폭 수 > 전체 데이터 셋 반복 수를 결정함 점점 많은 에폭 수를 두어 세밀하게 학습, 과적합 방지
# layers > 새로운 데이터셋에 맞추기 위해 heads만 / all의 경우 전체를 세밀하게 조정

# 초반에는 학습률을 크게 두고 적은 에폭 수로 빠르게 학습하고
# 뒤로 갈 수록 학습률을 촘촘하게 두고 많은 에폭 수로 세밀하고 최적의 학습을 찾는다

%%time
model.train(train_dataset, valid_dataset,
            learning_rate=LR/5, # 5분의 1로 낮춤
            epochs=EPOCHS[2],
            layers='all',
            augmentation=augmentation)

# 병합 과정
new_history = model.keras_model.history.history
for k in new_history: history[k] = history[k] + new_history[k]

In [ ]:
# 리스트의 마지막 값을 기준으로 에포크 범위를 설정
epochs = range(EPOCHS[-1])

plt.figure(figsize=(18, 6))

# 숫자는 행,열,위치 순서로 의미가 있음 > 131
plt.subplot(131)
# x축, y축, 선 이름
plt.plot(epochs, history['loss'], label="train loss")
plt.plot(epochs, history['val_loss'], label="valid loss")
plt.legend() # 범례 추가
plt.subplot(132)
plt.plot(epochs, history['mrcnn_class_loss'], label="train class loss")
plt.plot(epochs, history['val_mrcnn_class_loss'], label="valid class loss")
plt.legend()
plt.subplot(133)
plt.plot(epochs, history['mrcnn_mask_loss'], label="train mask loss")
plt.plot(epochs, history['val_mrcnn_mask_loss'], label="valid mask loss")
plt.legend()

plt.show()

In [ ]:
# history["val_loss"] 배열에서 가장 작은 값을 가진 인덱스 찾음 > argmin 기능 / 에폭 수 값을 알기 위해 +1 해줌
best_epoch = np.argmin(history["val_loss"]) + 1
print("Best epoch: ", best_epoch)
# 위에서 에폭 수를 구하기 위해 +1을 했기에 index 값을 찾기 위한 -1을 해줌
print("Valid loss: ", history["val_loss"][best_epoch-1])

In [ ]:
# 여기 코드는 수정해야 함 위쪽에서부터 읽지 못해서 그대로 뒀음
glob_list = glob.glob(f'/kaggle/working/fashion*/mask_rcnn_fashion_{best_epoch:04d}.h5')
model_path = glob_list[0] if glob_list else ''

In [ ]:
# 사전 학습된 가중치 로드
class InferenceConfig(FashionConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

# 추론 설정 적용
inference_config = InferenceConfig()

# 모델을 추론 모드로 설정함 > inference / model_dir > 저장할 공간
model = modellib.MaskRCNN(mode='inference', # training 모드도 있음
                          config=inference_config,
                          model_dir=ROOT_DIR)

# model_path 가 비어 있지 않은지 확인
assert model_path != '', "Provide path to trained weights"
print("Loading weights from ", model_path)
# 사전 학습된 가중치 로드
model.load_weights(model_path, by_name=True)

In [ ]:
sample_df = pd.read_csv(DATA_DIR/"sample_submission.csv")
sample_df.head()

In [ ]:
# 위에서 설명한 RLE 방식을 사용하여 데이터 압축
# 0과 1로 이루어진 리스트
def to_rle(bits):
    rle = [] # 저장할 리스트
    pos = 0 # bits 리스트에서 현재 처리 중인 위치를 나타냄
    # itertools.groupby를 통해 연속적인 동일 값을 그룹화 한다 + 값이 바뀔 때마다 새로운 그룹을 시작 함수 자체에서 처리
    # 그룹화 된 값을 bit는 그룹의 값, group은 그룹에 속한 값을 저장
    for bit, group in itertools.groupby(bits):
        group_list = list(group) # group 값을 리스트로 변환
        if bit: # 1일 경우
            rle.extend([pos, sum(group_list)]) # pos를 통해 얻은 현재 위치와 그룹의 길이를 rle에 추가
        pos += len(group_list)
    return rle

## 예시

bits = [1, 1, 0, 0, 1, 0]
for bit, group in itertools.groupby(bits):
    print(bit, list(group))

출력
1 [1, 1]
0 [0, 0]
1 [1]
0 [0]

itertools에 대해

groupby(iterable, key=None):
연속된 동일한 값을 그룹으로 묶습니다. 예를 들어, [1, 1, 0, 0, 1, 0]을 그룹화하여 [1, 1], [0, 0], [1], [0]로 나눕니다.

count(start=0, step=1):
start부터 step만큼 증가하는 무한 반복자를 생성합니다.

cycle(iterable):
주어진 반복 가능한 객체를 무한히 반복합니다.

repeat(object, times=None):
주어진 객체를 times 횟수만큼 반복합니다. times가 지정되지 않으면 무한히 반복합니다.

chain(*iterables):
여러 반복 가능한 객체들을 순차적으로 연결하여 하나의 반복자로 만듭니다.

islice(iterable, start, stop, step):
지정된 범위의 요소들을 슬라이스하여 반환합니다.

combinations(iterable, r):
주어진 반복 가능한 객체에서 길이 r인 모든 조합을 생성합니다.

permutations(iterable, r=None):
주어진 반복 가능한 객체에서 길이 r인 모든 순열을 생성합니다.

In [ ]:
# 마스크가 겹치지 않고 정리, 경계를 계산하여 정확도 향상
# masks는 3차원 배열로, 형태는 [height, width, num_masks]

def refine_masks(masks, rois):
    # masks.reshape(-1, masks.shape[-1]) > [height * width, num_masks] 형태로 배열 변환
    # reshape 배열의 형태 변경 함수 -1 로 자동으로 적절한 크기 결정
    areas = np.sum(masks.reshape(-1, masks.shape[-1]), axis=0)
    # 쉽게 설명하면 배열을 오름차순으로 정리하고 기존 인덱스 값을 출력함
    mask_index = np.argsort(areas) # 정렬을 인덱스 값으로 함 / 배열을 오름차순으로 정렬했을 때의 인덱스를 반환

    # masks.shape은 (height, width, num_masks) 형태의 튜플을 반환 -1을 통해 height, widet 형태의 배열
    union_mask = np.zeros(masks.shape[:-1], dtype=bool) # 빈 마스크 초기화

    # 작은 면적의 마스크부터 순서대로 겹치는 부분을 제거
    for m in mask_index:
        # union_mask에 not을 취하고 m번째 마스크와 AND 연산 > 겹치지 않는 부분만 남김
        masks[:, :, m] = np.logical_and(masks[:, :, m], np.logical_not(union_mask))
        # union_mask 현재까지 합쳐진 마스크와 m번 째 마스크 OR 연산
        union_mask = np.logical_or(masks[:, :, m], union_mask)

    # ROI 업데이트
    for m in range(masks.shape[-1]):
        # m번째가 True면 행,열 반환
        mask_pos = np.where(masks[:, :, m]==True)

        # 마스크의 경계 부분을 파악하고 객체의 위치와 크기를 파악할 수 있음 > min, max를 통해
        if np.any(mask_pos): # True인 경우에만 확인
            # 경계에 대한 부분을 나타내는 좌표
            y1, x1 = np.min(mask_pos, axis=1)
            y2, x2 = np.max(mask_pos, axis=1)
            rois[m, :] = [y1, x1, y2, x2]
    return masks, rois

In [ ]:
%%time
sub_list = [] # 예측 결과 저장
missing_count = 0 # 마스크가 없는 이미지 카운트

# tqdm > 실시간 확인 가능 / iterrows 를 통해 데이터 프레임 행을 하나씩 반복하여 처리
# sample_df.iterrows() > 데이터 프레임의 각 행을 순차적으로 반환
for i, row in tqdm(sample_df.iterrows(), total=len(sample_df)):
    # 이미지 로드하고 리사이즈
    image = resize_image(str(DATA_DIR/'test'/row['ImageId']))
    # 리사이즈 된 이미지를 모델에 입력 / model.detect 함수로 예측 결과 반환
    # detect를 통해 여러 이미지를 처리할 수 있고 리스트 형태로 반환된다
    # [image]에서 단일 이미지에 대한 예측 결과 리스트
    result = model.detect([image])[0

    # 마스크가 있는 경우 처리
    if result['masks'].size > 0:
        # 객체들의 마스크, 객체들의 경계 / _ 는 일반적으로 중요하지 않아서 무시할 때 사용
        # masks, rois 2가지 값을 반환 받는데 코드에서 rois 사용하지 않음
        masks, _ = refine_masks(result['masks'], result['rois'])
        for m in range(masks.shape[-1]): # 마스크의 개수
            # mask는 3차원 배열(h,w,m) 모든행, 모든열, m번째 마스크
            # ravel 1차원으로 펼치는 함수 / F > 열 우선 순서로 배열
            mask = masks[:, :, m].ravel(order='F')
            rle = to_rle(mask)  # to_rle 함수
            label = result['class_ids'][m] - 1
            # 현재 이미지의 아이디, RLE 리스트를 문자열로 반환, 아래 예시 찾모
            sub_list.append([row['ImageId'], ' '.join(list(map(str, rle))), label])
    else:
        # 마스크가 없으면 RLE 기본 값 '1 1', 기본 클래스 ID 23
        sub_list.append([row['ImageId'], '1 1', 23])
        missing_count += 1

## 예시

data = {'A': [1, 2, 3], 'B': [4, 5, 6]}
sample_df = pd.DataFrame(data)

### iterrows와 tqdm 사용 예시
for i, row in tqdm(sample_df.iterrows(), total=len(sample_df)):
    print(i, row)

0 A    1
  B    4
Name: 0, dtype: int64
1 A    2
  B    5
Name: 1, dtype: int64
2 A    3
  B    6
Name: 2, dtype: int64


### ' '.join(list(map(str, rle)))
to_rle(mask)는 마스크를 RLE (Run-Length Encoding) 형태로 변환합니다.
예를 들어, [1, 1, 0, 0, 1, 0]은 [1 2, 5 1]로 변환됩니다.

map(str, rle)는 map 함수를 통해 RLE 리스트의 각 요소를 순서대로 가져와 str을 통해 문자열로 변환합니다.
rle 리스트의 각 숫자 요소를 문자열로 변환합니다.

list(map(str, rle))는 문자열 리스트를 만듭니다.
예를 들어, ['1', '2', '5', '1']로 변환됩니다.

' '.join(list(map(str, rle)))는 이 문자열 리스트를 하나의 문자열로 결합합니다.
' '.join(['1', '2', '5', '1'])는 "1 2 5 1"로 변환됩니다.
이렇게 하는 이유는 최종적으로 제출할 때 문자열 형태의 RLE가 필요하기 때문입니다.

In [ ]:
# 예측 결과를 데이터 프레임으로 변환 / sample_df.columns.values > 열 설정
submission_df = pd.DataFrame(sub_list, columns=sample_df.columns.values)
print("Total image results: ", submission_df['ImageId'].nunique())
print("Missing Images: ", missing_count) # 마스크가 없는 이미지 수
submission_df.head()

In [ ]:
submission_df.to_csv("submission.csv", index=False)

In [ ]:
for i in range(9):
    # 무작위 하나의 이미지 ID
    image_id = sample_df.sample()['ImageId'].values[0]
    image_path = str(DATA_DIR/'test'/image_id)
    
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    result = model.detect([resize_image(image_path)])
    r = result[0]
    
    if r['masks'].size > 0:
        masks = np.zeros((img.shape[0], img.shape[1], r['masks'].shape[-1]), dtype=np.uint8)
        for m in range(r['masks'].shape[-1]):
            masks[:, :, m] = cv2.resize(r['masks'][:, :, m].astype('uint8'), 
                                        (img.shape[1], img.shape[0]), interpolation=cv2.INTER_NEAREST)
        
        y_scale = img.shape[0]/IMAGE_SIZE
        x_scale = img.shape[1]/IMAGE_SIZE
        rois = (r['rois'] * [y_scale, x_scale, y_scale, x_scale]).astype(int)
        
        masks, rois = refine_masks(masks, rois)
    else:
        masks, rois = r['masks'], r['rois']
        
    visualize.display_instances(img, rois, masks, r['class_ids'], 
                                ['bg']+label_names, r['scores'],
                                title=image_id, figsize=(12, 12))